In [1]:
from collections import Iterable
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from time import sleep
import selenium.webdriver.support.expected_conditions as EC
import pandas as pd

%load_ext nb_black

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


<IPython.core.display.Javascript object>

In [2]:
browser = webdriver.Chrome(r"E:/BaiduYunDownload/chromedriver")
browser.get(r"http://data.cnki.net/ValueSearch/Index?datatype=year&ky=GDP")

<IPython.core.display.Javascript object>

In [3]:
def find_table(indicator, region, year):

    browser.find_element_by_name("IndicateName").clear()
    browser.find_element_by_name("IndicateName").send_keys(indicator)
    browser.find_element_by_name("IndicateRegion").clear()
    browser.find_element_by_name("IndicateRegion").send_keys(region)
    browser.find_element_by_name("StartYear").send_keys(year)
    browser.find_element_by_name("EndYear").send_keys(year)
    element = browser.find_element_by_id("AdvancedSearch")
    browser.execute_script(
        "arguments[0].click();", element
    )  # arguments[0], element means passing element into arguments[0] to execute in Javascript
    xpath = "/html/body/div[1]/div[3]/div[1]/div/div[2]/table/tbody/tr"
    sleep(20)

    ls = []
    try:
        table = browser.find_elements_by_xpath(xpath)
        for tr in table:
            tds = tr.find_elements_by_tag_name("td")
            ls.append([td.text for td in tds])
    except EC.StaleElementReferenceException:
        table = browser.find_elements_by_xpath(xpath)
        for tr in table:

            tds = tr.find_elements_by_tag_name("td")
            ls.append([td.text for td in tds])
    table_df = pd.DataFrame(ls)
    return table_df

<IPython.core.display.Javascript object>

In [4]:
def data_collect(
    indicator_list, region_list, year_range, year_filter=None, unit_filter=None
):
    city_ls = []
    year_ls = []
    indicator_ls = []
    for city in region_list:
        for year in year_range:
            for indicator in indicator_list:
                indicator_ls.append(find_table(indicator, city, str(year)))
            indicator_data = pd.concat(indicator_ls)
            year_ls.append(indicator_data)
        year_data = pd.concat(year_ls)
        city_ls.append(year_data)
    total_df = pd.concat(city_ls)
    try:
        if isinstance(year_filter, Iterable):
            total_df_screened = total_df[
                (total_df[2].str.contains("|".join(year_filter)))
                & (total_df[6].str.contains(unit_filter))
            ].iloc[:, 2:7]
        else:
            total_df_screened = total_df[
                (total_df[2].str.contains(year_filter))
                & (total_df[6].str.contains(unit_filter))
            ].iloc[:, 2:7]
    except:
        total_df_screened = total_df
    return total_df_screened

<IPython.core.display.Javascript object>

In [5]:
year_range = list(range(2005, 2019))
year_range

[2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

<IPython.core.display.Javascript object>

In [6]:
year_filter = [str(i) for i in year_range]
year_filter

['2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018']

<IPython.core.display.Javascript object>

In [7]:
province = [
    "北京",
    "天津",
    "河北",
    "山西",
    "内蒙古",
    "辽宁",
    "吉林",
    "黑龙江",
    "上海",
    "江苏",
    "浙江",
    "安徽",
    "福建",
    "江西",
    "山东",
    "河南",
    "湖北",
    "湖南",
    "广东",
    "广西",
    "海南",
    "重庆",
    "四川",
    "贵州",
    "云南",
    "陕西",
    "甘肃",
    "青海",
    "宁夏",
    "新疆",
]

<IPython.core.display.Javascript object>

In [8]:
indicator_list = ["能源消费总量"]

<IPython.core.display.Javascript object>

In [9]:
data_df = data_collect(
    indicator_list, province, year_range, year_filter=year_filter, unit_filter="标准煤"
)

<IPython.core.display.Javascript object>

In [10]:
data_df

,2,3,4,5,6
0,2005年,北京市,能源、资源和环境能源消费总量,5049.8,万吨标准煤
1,2005年,北京市,能源消费总量,5049.8,万吨标准煤
2,2005年,北京市,第三产业能源消费总量,1771.7,万吨标准煤
8,2005年,北京市,"京津冀能源消费总量,能源消费总量",5521.9,万吨标准煤
18,2005年,北京,能源消费总量,5522,万吨标准煤
...,...,...,...,...,...
9,2017年,新疆,可供消费的能源总量,16058.47,万吨标准煤
10,2017年,新疆,能源消费总量,17391.7,万吨标准煤
11,2017年,新疆,工业终端能源消费总量,11826.93,万吨标准煤
12,2017年,新疆,能源消费总量中损失量,230.08,万吨标准煤


<IPython.core.display.Javascript object>

In [11]:
data_df_screened = data_df.drop_duplicates()

<IPython.core.display.Javascript object>

In [12]:
data_df_screened

,2,3,4,5,6
0,2005年,北京市,能源、资源和环境能源消费总量,5049.8,万吨标准煤
1,2005年,北京市,能源消费总量,5049.8,万吨标准煤
2,2005年,北京市,第三产业能源消费总量,1771.7,万吨标准煤
8,2005年,北京市,"京津冀能源消费总量,能源消费总量",5521.9,万吨标准煤
18,2005年,北京,能源消费总量,5522,万吨标准煤
...,...,...,...,...,...
1,2017年,新疆,能源消费总量,17391.7,万吨标准煤
9,2017年,新疆,可供消费的能源总量,16058.47,万吨标准煤
11,2017年,新疆,工业终端能源消费总量,11826.93,万吨标准煤
12,2017年,新疆,能源消费总量中损失量,230.08,万吨标准煤


<IPython.core.display.Javascript object>

In [13]:
path = (
    r"E:\tencent files\chrome Download\Research\DEA\DEA_carbon market\Data_collection\\"
)

<IPython.core.display.Javascript object>

In [14]:
data_df_screened.to_excel(path + indicator_list[0] + ".xlsx")

<IPython.core.display.Javascript object>